In [1]:
#all imports
import os
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import sys
import random
import math
sys.path.append('/opt/carla-simulator/PythonAPI/carla') # tweak to where you put carla
from keras.models import load_model
from agents.navigation.global_route_planner import GlobalRoutePlanner
from matplotlib import pyplot as plt 

# disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

2025-01-23 12:44:17.296345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-23 12:44:17.820260: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-23 12:44:19.808513: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/winter/.pyenv/versions/3.7.17/envs/carla-0.9.13-py3.7/lib/python3.7/site-packages

In [2]:
client = carla.Client('10.8.179.139', 2000)
# start a car
world = client.get_world()

#clean up
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [3]:
# Define basic settings
PREFERRED_SPEED = 30
SPEED_THRESHOLD = 2 # defines when we get close to desired speed so we drop the speed

# Max steering angle
MAX_STEER_DEGREES = 40
# This is max actual angle with Mini under steering input=1.0
STEERING_CONVERSION = 75

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

# resize images before running thgem through the model
# this is the same as when yo train the model
HEIGHT = 66
WIDTH = 200

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

In [31]:
# utility function for camera listening 
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

# utility function for camera listening 
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 


# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees

def get_proper_angle(car,wp_idx,rte):
    '''
    This function uses simple fuction above to get angle but for current
    waypoint and a few more next waypoints to ensure we have not skipped
    next waypoint so we avoid the car trying to turn back
    '''
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(10):
        if wp_idx + i*3 <len(rte)-1:
            next_angle_list.append(get_angle(car,rte[wp_idx + i*3][0]))
    idx = 0
    while idx<len(next_angle_list)-2 and abs(next_angle_list[idx])>40:
        idx +=1
    return wp_idx+idx*3,next_angle_list[idx]  


def draw_route(wp, route,seconds=3.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route

def exit_clean():
    #clean up
    cv2.destroyAllWindows()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None

def predict_angle(RGB_im):
    img = np.float32(RGB_im)
    img = img /255              # normalize
    img = np.expand_dims(img, axis=0)
    angle = model(img,training=False)
    return angle.numpy()[0][0] / math.pi


# spawn the car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Tesla m3 car
vehicle_bp = world.get_blueprint_library().filter('*model3*')

# load Nvidia DAVE-2 model
MODEL_NAME = 'model/model.h5'
model = load_model(MODEL_NAME,compile=False)
model.compile()
quit = False

In [41]:
# main loop
while True:
    start_point = random.choice(spawn_points)
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
    #setting RGB Camera - this follow the approach explained in a Carla video
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', '360')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()

    camera_data = {'image': np.zeros((image_h,image_w,3))} # 3 channels for RGB

    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))
    cv2.namedWindow('RGB Camera',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('RGB Camera', 800, 450)
    cv2.imshow('RGB Camera',camera_data['image'])
    # getting a random route for the car
    route = select_random_route(start_point,spawn_points)
    curr_wp = 5 #we will be tracking waypoints in the route and switch to next one when we get close to current one
    predicted_angle = 0
    smoothed_angle = 0
    PREFERRED_SPEED = 30 # setting speed at start of new route
    
    spectator = world.get_spectator()
    spectator_pos = carla.Transform(start_point.location + carla.Location(x=-20,y=10,z=10),
                                carla.Rotation(yaw = start_point.rotation.yaw -155))
    spectator.set_transform(spectator_pos)

    while curr_wp<len(route)-1:
        # Carla Tick
        world.tick()
        draw_route(curr_wp, route,1)
        if cv2.waitKey(1) == ord('q'):
            quit = True
            exit_clean()
            break
        image = camera_data['image']
        image_show = image
        image = cv2.resize(image, (WIDTH,HEIGHT))


        # Spectator Update
        spectator_transform = vehicle.get_transform()
        spectator_transform.location += carla.Location(x=0, y=0, z=15)
        spectator_transform.rotation.yaw += -15  # left
        spectator_transform.rotation.pitch = -60 # downward
        spectator.set_transform(spectator_transform)
        
        
        if curr_wp >=len(route)-10: # within 10 points of end, the route is done
            PREFERRED_SPEED = 0 # seeting speed to 0 after completing one route
            exit_clean()
            break
        while curr_wp<len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
            curr_wp +=1 #move to next wp if we are too close
        curr_wp, predicted_angle = get_proper_angle(vehicle,curr_wp,route)
        
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)

        estimated_throttle = maintain_speed(speed)
        # use the model to predict steering - predictions are expected to be in -1 to +1
        steer_input = predict_angle(image)
        smoothed_angle += 0.2 * pow(abs((steer_input - smoothed_angle)), 2.0 / 3.0) * (steer_input - smoothed_angle) / abs(steer_input - smoothed_angle)

        
        vehicle.apply_control(carla.VehicleControl(throttle=float(estimated_throttle), steer=float(steer_input)))

        image_show = cv2.UMat(image_show)
        image_show = cv2.putText(image_show, 'Speed: '+str(int(speed))+' kmh', org, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
        image_show = cv2.putText(image_show, 'Actual Angle: '+ str(int(predicted_angle)), org2, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
        image_show = cv2.putText(image_show, 'Model predict: '+str(float(smoothed_angle)), org3, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
        print("Model predict: %.3f \t Actual Angle: %f" % (smoothed_angle * 180, predicted_angle))
        cv2.imshow('RGB Camera', cv2.resize(image_show, (800, 398)))
    if quit:
        break

Model predict: -6.577 	 Actual Angle: -24.443637
Model predict: -10.892 	 Actual Angle: -24.443637
Model predict: -13.174 	 Actual Angle: -24.443637
Model predict: -14.148 	 Actual Angle: -24.443637
Model predict: -13.555 	 Actual Angle: -24.443637
Model predict: -14.252 	 Actual Angle: -24.443637
Model predict: -13.861 	 Actual Angle: -24.443637
Model predict: -13.612 	 Actual Angle: -24.443637
Model predict: -13.834 	 Actual Angle: -24.443637
Model predict: -13.641 	 Actual Angle: -24.443637
Model predict: -13.865 	 Actual Angle: -24.443637
Model predict: -14.096 	 Actual Angle: -24.443637
Model predict: -13.368 	 Actual Angle: -24.443637
Model predict: -14.184 	 Actual Angle: -24.444088
Model predict: -13.419 	 Actual Angle: -24.468626
Model predict: -14.985 	 Actual Angle: -24.511830
Model predict: -13.489 	 Actual Angle: -24.550364
Model predict: -13.750 	 Actual Angle: -24.608224
Model predict: -14.275 	 Actual Angle: -24.678822
Model predict: -14.654 	 Actual Angle: -24.766117
M